#### Model Specification
Model: scratch resnet56 | Data: 30 classes | Dropout:  | Max Size:  | Remarks: Using techniques of superconvergence and follow steps in tutorial

Result: 

Issues: Cudo out of memory

Reference: https://github.com/sgugger/Deep-Learning/blob/master/Cyclical%20LR%20and%20momentums.ipynb

[24/01/2019]

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('/home/r8user2/fastai/old')
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
import matplotlib.pyplot as plt

In [ ]:
PATH = '/home/r8user2/Documents/HY/dress_data/alex_workplace/dressdata_project/git_workplace/selected_gd_data_30'
CPU_WORKERS = 60
sz = 224
# stats may need to adjust
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

In [ ]:
def get_data(sz,bs = 64):
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()])
    return ImageClassifierData.from_paths(PATH, 
                                          trn_name = 'imgtrain',
                                          val_name = 'imgval',
                                          test_name = 'imgtest',
                                          test_with_labels = True,
                                          num_workers = CPU_WORKERS,
                                          tfms=tfms, 
                                          bs=bs)

In [ ]:
data = get_data(sz)

### Build Resnet56 From Scratch

In [ ]:
class BasicBlock(nn.Module):
    
    def __init__(self, ch_in, ch_out, stride=1):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(ch_out)
        self.conv1 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(ch_out)
        self.conv2 = nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1, bias=False)

        if stride != 1 or ch_in != ch_out:
            self.shortcut = nn.Sequential(
                nn.Conv2d(ch_in, ch_out, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(ch_out)
            )

    def forward(self, x):
        shortcut = self.shortcut(x) if hasattr(self, 'shortcut') else x
        out = self.conv1(x)
        out = self.bn2(self.conv2(F.relu(self.bn1(out))))
        out += shortcut
        return F.relu(out)

In [ ]:
# Change num_classes!
class ResNet(nn.Module):
    def __init__(self, num_blocks, num_classes=30):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self.make_group_layer(16, 16, num_blocks[0], stride=1)
        self.layer2 = self.make_group_layer(16, 32, num_blocks[1], stride=2)
        self.layer3 = self.make_group_layer(32, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

    def make_group_layer(self,ch_in, ch_out, num_blocks, stride):
        layers = [BasicBlock(ch_in, ch_out, stride)]
        for i in range(num_blocks-1):
            layers.append(BasicBlock(ch_out, ch_out, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        return F.log_softmax(self.linear(out))

In [ ]:
learn = ConvLearner.from_model_data(ResNet([9,9,9]), data)

### Learning Rate Finder

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()

### Fine Tune Weight Decays

In [ ]:
def fix_wds(wds):
    learn = ConvLearner.from_model_data(ResNet([9,9,9]), data)
    learn.crit = F.nll_loss
    learn.lr_find2(wds = wds,start_lr = 1e-4, end_lr = 100,num_it = 100)
    trn_loss = learn.sched.losses
    lrsw = learn.sched.lrs
    return trn_loss, lrsw

In [ ]:
trn_loss1, lrsw1 = fix_wds(wds = 1e-2)